Clientes recorrentes.

In [ ]:
# @title
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df_orders = pd.read_csv('Order_semicolon.csv', sep=';')

customer_orders = df_orders['customer'].value_counts()

new_customers = customer_orders[customer_orders == 1].index
returning_customers = customer_orders[customer_orders > 1].index

total_customers = len(customer_orders)
num_new_customers = len(new_customers)
percentage_new = (num_new_customers / total_customers) * 100

print("### Análise de Novos Clientes ###")
print(f"Total de clientes: {total_customers}")
print(f"Clientes novos com 1 pedido: {num_new_customers}")
print(f"Clientes recorrentes com 1 ou mais pedidos: {len(returning_customers)}")
print(f"Porcentagem de clientes novos: {percentage_new:.2f}%\n")

multi_order_customers = customer_orders[customer_orders >= 3].index

if not multi_order_customers.empty:
    print("### Análise de Clientes com 3+ Pedidos ###")

    top_customers_info = []

    for customer_id in multi_order_customers:

      customer_df = df_orders[df_orders['customer'] == customer_id]
      most_common_channel = customer_df['salesChannel'].mode()[0]

      top_customers_info.append({
            'Cliente': customer_id,
            'Total de Pedidos': len(customer_df),
            'Canal de Venda Mais Usado': most_common_channel
        })

      top_customers_df = pd.DataFrame(top_customers_info).sort_values(
        by='Total de Pedidos', ascending=False
    ).reset_index(drop=True)

    print(top_customers_df)

else:
    print("Não foram encontrados clientes com 3 ou mais pedidos.")

sns.set_style("whitegrid")

plt.figure(figsize=(8, 5))
customer_orders.value_counts().sort_index().plot(
    kind='bar',
    color='skyblue',
    edgecolor='black'
)

plt.title('Distribuição de Clientes por Número de Pedidos', fontsize=14)
plt.xlabel('Número de Pedidos por Cliente', fontsize=12)
plt.ylabel('Número de Clientes', fontsize=12)
plt.xticks(rotation=0)

for i, v in enumerate(customer_orders.value_counts().sort_index()):
    plt.text(i, v + 0.1, str(v), ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()



if not top_customers_df.empty:

    channel_counts = top_customers_df['Canal de Venda Mais Usado'].value_counts()

    plt.figure(figsize=(7, 5))
    channel_counts.plot(
        kind='bar',
        color=['#4CAF50', '#FF9800', '#2196F3', '#F4436E'],
        edgecolor='black'
    )

    plt.title('Canal de Venda Mais Usado por Clientes com 3+ Pedidos', fontsize=14)
    plt.xlabel('Canal de Venda', fontsize=12)
    plt.ylabel('Contagem de Clientes', fontsize=12)
    plt.xticks(rotation=0)

    for i, v in enumerate(channel_counts):
        plt.text(i, v + 0.1, str(v), ha='center', va='bottom', fontsize=10)

    plt.tight_layout()
    plt.show()

In [ ]:
# @title
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings("ignore", category=UserWarning, module="pandas")

sns.set_style("whitegrid")
pd.set_option('display.max_columns', None)

order = pd.read_csv("Order_semicolon.csv", sep=";")
campaign = pd.read_csv("Campaign_semicolon.csv", sep=";")
campaign_queue = pd.read_csv("CampaignQueue_semicolon.csv", sep=";")

print(f"Order: {order.shape} | Campaign: {campaign.shape} | CampaignQueue: {campaign_queue.shape}\n")

order = order[['id','customer','customerAge','customerRegion','salesChannel','status','totalAmount','createdAt']]
campaign = campaign[['id','name','badge','type','status','storeId','createdAt','updatedAt']]
campaign_queue = campaign_queue[['id','campaignId','storeId','customerId','scheduledAt','sendAt','status','createdAt']]

for df in [order, campaign, campaign_queue]:
    df.columns = df.columns.str.lower()

def converter_datas_seguras(df):
    for col in df.columns:
        if "at" in col.lower():
            exemplo = df[col].dropna().astype(str).iloc[0] if not df[col].dropna().empty else ''
            if '-' in exemplo and exemplo[:4].isdigit():
                fmt = "%Y-%m-%d %H:%M:%S" if " " in exemplo else "%Y-%m-%d"
                df[col] = pd.to_datetime(df[col], format=fmt, errors='coerce')
            elif '/' in exemplo:
                fmt = "%d/%m/%Y %H:%M" if ":" in exemplo else "%d/%m/%Y"
                df[col] = pd.to_datetime(df[col], format=fmt, errors='coerce', dayfirst=True)
            else:
                df[col] = pd.to_datetime(df[col], errors='coerce')
    return df

order = converter_datas_seguras(order)
campaign = converter_datas_seguras(campaign)
campaign_queue = converter_datas_seguras(campaign_queue)

order.drop_duplicates(inplace=True)
campaign.drop_duplicates(inplace=True)
campaign_queue.drop_duplicates(inplace=True)

order.dropna(subset=['customer','totalamount'], inplace=True)
campaign_queue.dropna(subset=['customerid'], inplace=True)

order['ano_pedido'] = order['createdat'].dt.year
order['mes_pedido'] = order['createdat'].dt.month
order['valor_categoria'] = pd.cut(order['totalamount'],
                                  bins=[0, 50, 100, 200, 1000],
                                  labels=['baixo','médio','alto','premium'])

order['faixa_etaria'] = pd.cut(order['customerage'],
                               bins=[17,25,35,50,65],
                               labels=['18-25','26-35','36-50','51-65'])

campaign['dias_ativo'] = (campaign['updatedat'] - campaign['createdat']).dt.days
campaign_queue['tempo_envio_horas'] = (campaign_queue['sendat'] - campaign_queue['scheduledat']).dt.total_seconds() / 3600

integrado = (
    campaign_queue
    .merge(campaign, left_on='campaignid', right_on='id', suffixes=('_queue','_camp'))
    .merge(order, left_on='customerid', right_on='customer', how='left')
)

print(f"Total de registros: {integrado.shape[0]}\n")

integrado.columns = [col.lower() for col in integrado.columns]
integrado['totalamount'] = integrado['totalamount'].astype(float).round(2)
integrado.to_csv("dados_integrados_limpos.csv", sep=';', index=False)

print(" Arquivo: dados_integrados_limpos.csv\n")

df_orders = order.copy()
customer_orders = df_orders['customer'].value_counts()

new_customers = customer_orders[customer_orders == 1].index
returning_customers = customer_orders[customer_orders > 1].index

total_customers = len(customer_orders)
num_new_customers = len(new_customers)
percentage_new = (num_new_customers / total_customers) * 100

print("Análise de Clientes Novos")
print(f"Total de clientes: {total_customers}")
print(f"Clientes novos com 1 pedido: {num_new_customers}")
print(f"Clientes recorrentes (2+ pedidos): {len(returning_customers)}")
print(f"Porcentagem de clientes novos: {percentage_new:.2f}%\n")

multi_order_customers = customer_orders[customer_orders >= 3].index

if not multi_order_customers.empty:
    top_customers_info = []
    for customer_id in multi_order_customers:
        customer_df = df_orders[df_orders['customer'] == customer_id]
        most_common_channel = customer_df['saleschannel'].mode()[0]
        top_customers_info.append({
            'Cliente': customer_id,
            'Total de Pedidos': len(customer_df),
            'Canal de Venda Mais Usado': most_common_channel
        })
    top_customers_df = pd.DataFrame(top_customers_info).sort_values(
        by='Total de Pedidos', ascending=False).reset_index(drop=True)
    print("### Clientes com 3 ou mais Pedidos ###")
    print(top_customers_df)
else:
    print("Não há clientes com 3 ou mais pedidos.")

plt.figure(figsize=(8, 5))
customer_orders.value_counts().sort_index().plot(
    kind='bar', color='skyblue', edgecolor='black'
)
plt.title('Distribuição de Clientes por Número de Pedidos', fontsize=14)
plt.xlabel('Número de Pedidos por Cliente', fontsize=12)
plt.ylabel('Número de Clientes', fontsize=12)
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

if 'top_customers_df' in locals() and not top_customers_df.empty:
    channel_counts = top_customers_df['Canal de Venda Mais Usado'].value_counts()
    plt.figure(figsize=(7, 5))
    channel_counts.plot(kind='bar', color=['#4CAF50','#FF9800','#2196F3','#F44336'], edgecolor='black')
    plt.title('Canal de Venda Mais Usado por Clientes com 3+ Pedidos', fontsize=14)
    plt.xlabel('Canal de Venda', fontsize=12)
    plt.ylabel('Contagem de Clientes', fontsize=12)
    plt.xticks(rotation=0)
    plt.tight_layout()
    plt.show()


10 Campanhas com mais pedidos.

In [ ]:
# @title

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df_pedidos = pd.read_csv('Order_semicolon.csv', sep=';')
df_campanhas = pd.read_csv('CampaignQueue_semicolon.csv', sep=';')

pedidos_por_cliente = df_pedidos['customer'].value_counts().reset_index()
pedidos_por_cliente.columns = ['customer_id', 'total_pedidos']

df_analise = pd.merge(df_campanhas, pedidos_por_cliente, left_on='customerId', right_on='customer_id', how='left')

df_analise['total_pedidos'] = df_analise['total_pedidos'].fillna(0).astype(int)

total_pedidos_por_campanha = (
    df_analise.groupby('campaignId')['total_pedidos']
    .sum()
    .sort_values(ascending=False)
    .reset_index()
)

print("Quantidade Total de Pedidos por Campanha")
print("-" * 60)
print(total_pedidos_por_campanha.head(10).to_string(index=False))

df_top_10 = total_pedidos_por_campanha.head(10)

sns.set_theme(style="whitegrid")

plt.figure(figsize=(10, 6))
ax = sns.barplot(
    x='total_pedidos',
    y='campaignId',
    hue='campaignId',
    data=df_top_10,
    palette='viridis',
    legend=False
)

plt.title('Top 10 Campanhas por Quantidade Total de Pedidos', fontsize=16)
plt.xlabel('Total de Pedidos Gerados', fontsize=12)
plt.ylabel('ID da Campanha', fontsize=12)

for index, row in df_top_10.iterrows():
    ax.text(row['total_pedidos'] + 0.1, index, str(row['total_pedidos']),
            color='black', ha="left", va="center", fontsize=10)

plt.subplots_adjust(left=0.25, right=0.95, top=0.9, bottom=0.1)
plt.show()


In [ ]:
# @title
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid")
pd.set_option('display.max_columns', None)

print("Iniciando a preparação dos dados\n")

df_pedidos = pd.read_csv('Order_semicolon.csv', sep=';')
df_campanhas = pd.read_csv('CampaignQueue_semicolon.csv', sep=';')

print(f"Pedidos: {df_pedidos.shape} | Campanhas: {df_campanhas.shape}\n")

df_pedidos = df_pedidos[['id', 'customer', 'customerRegion', 'totalAmount']]
df_campanhas = df_campanhas[['id', 'campaignId', 'customerId', 'status']]

df_pedidos.drop_duplicates(inplace=True)
df_campanhas.drop_duplicates(inplace=True)
df_pedidos.dropna(subset=['customer', 'customerRegion', 'totalAmount'], inplace=True)
df_pedidos['totalAmount'] = pd.to_numeric(df_pedidos['totalAmount'], errors='coerce').fillna(0)

print("Integrando pedidos e campanhas")

pedidos_por_cliente = df_pedidos['customer'].value_counts().reset_index()
pedidos_por_cliente.columns = ['customer_id', 'total_pedidos']

df_analise = pd.merge(df_campanhas, pedidos_por_cliente,
                      left_on='customerId', right_on='customer_id', how='left')

df_analise['total_pedidos'] = df_analise['total_pedidos'].fillna(0).astype(int)

print("Quantidade total de pedidos por campanha.\n")

total_pedidos_por_campanha = (
    df_analise.groupby('campaignId')['total_pedidos']
    .sum()
    .sort_values(ascending=False)
    .reset_index()
)

print("===== Quantidade Total de Pedidos por Campanha =====")
print(total_pedidos_por_campanha.head(10).to_string(index=False))
print("=====================================================\n")

df_top_10 = total_pedidos_por_campanha.head(10)

plt.figure(figsize=(10, 6))

ax = sns.barplot(
    x='total_pedidos',
    y='campaignId',
    hue='campaignId',
    data=df_top_10,
    palette='viridis',
    legend=False
)
plt.title('Top 10 Campanhas por Quantidade Total de Pedidos', fontsize=16)
plt.xlabel('Total de Pedidos Gerados', fontsize=12)
plt.ylabel('ID da Campanha', fontsize=12)

for index, row in df_top_10.iterrows():
    ax.text(row['total_pedidos'] + 0.1, index, str(row['total_pedidos']),
            color='black', ha="left", va="center", fontsize=10)
plt.subplots_adjust(left=0.25, right=0.95, top=0.9, bottom=0.1)

plt.show()

total_pedidos_por_campanha.to_csv('analise_pedidos_por_campanha.csv', sep=';', index=False)

print("Arquivo salvo: analise_pedidos_por_campanha.csv")

10 Campanhas maior média de pedidos.

In [ ]:
# @title
import pandas as pd
import matplotlib.pyplot as plt

df_pedidos = pd.read_csv('Order_semicolon.csv', sep=';')
df_campanhas = pd.read_csv('Campaign_semicolon.csv', sep=';')
df_fila_campanhas = pd.read_csv('CampaignQueue_semicolon.csv', sep=';')

pedidos_por_cliente = df_pedidos['customer'].value_counts().reset_index()
pedidos_por_cliente.columns = ['customerId', 'total_pedidos']

df_analise = pd.merge(df_fila_campanhas, pedidos_por_cliente, on='customerId', how='left')

df_analise['total_pedidos'] = df_analise['total_pedidos'].fillna(0)

media_por_campanha = df_analise.groupby('campaignId')['total_pedidos'].mean().reset_index()
media_por_campanha.columns = ['campaignId', 'media_pedidos']

df_nomes_campanhas = df_campanhas[['id', 'name']]
df_resultado = pd.merge(media_por_campanha, df_nomes_campanhas, left_on='campaignId', right_on='id', how='left')

df_resultado = df_resultado.sort_values(by='media_pedidos', ascending=False).head(10)

plt.figure(figsize=(12, 8))
plt.barh(df_resultado['name'], df_resultado['media_pedidos'], color='skyblue')
plt.title('Top 10 Campanhas com Maior Média de Pedidos por Cliente', fontsize=16)
plt.xlabel('Média de Pedidos por Cliente', fontsize=12)
plt.ylabel('Nome da Campanha', fontsize=12)
plt.grid(axis='x', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

In [ ]:
# @title
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8-whitegrid')

print("Análise de campanhas\n")

df_pedidos = pd.read_csv('Order_semicolon.csv', sep=';')
df_campanhas = pd.read_csv('Campaign_semicolon.csv', sep=';')
df_fila_campanhas = pd.read_csv('CampaignQueue_semicolon.csv', sep=';')

print(f"Dados Obtidos!")
print(f"Pedidos: {df_pedidos.shape} | Campanhas: {df_campanhas.shape} | Fila: {df_fila_campanhas.shape}\n")

df_pedidos = df_pedidos[['id', 'customer', 'totalAmount']]
df_campanhas = df_campanhas[['id', 'name']]
df_fila_campanhas = df_fila_campanhas[['id', 'campaignId', 'customerId', 'status']]

df_pedidos.drop_duplicates(inplace=True)
df_campanhas.drop_duplicates(inplace=True)
df_fila_campanhas.drop_duplicates(inplace=True)

df_pedidos.dropna(subset=['customer', 'totalAmount'], inplace=True)
df_fila_campanhas.dropna(subset=['customerId'], inplace=True)


df_pedidos['totalAmount'] = pd.to_numeric(df_pedidos['totalAmount'], errors='coerce').fillna(0)

print("Dados limpos e padronizados!\n")

pedidos_por_cliente = df_pedidos['customer'].value_counts().reset_index()
pedidos_por_cliente.columns = ['customerId', 'total_pedidos']
df_analise = pd.merge(df_fila_campanhas, pedidos_por_cliente, on='customerId', how='left')
df_analise['total_pedidos'] = df_analise['total_pedidos'].fillna(0)

print("Integração concluída!\n")

media_por_campanha = df_analise.groupby('campaignId')['total_pedidos'].mean().reset_index()
media_por_campanha.columns = ['campaignId', 'media_pedidos']

df_resultado = pd.merge(media_por_campanha, df_campanhas, left_on='campaignId', right_on='id', how='left')

df_resultado = df_resultado.sort_values(by='media_pedidos', ascending=False).head(10)

print("Média calculada com sucesso!")
print("===== Top 10 Campanhas com Maior Média de Pedidos =====")
print(df_resultado[['name', 'media_pedidos']].to_string(index=False))
print("========================================================\n")

print("📊 Gerando gráfico das 10 principais campanhas...\n")

plt.figure(figsize=(12, 8))
plt.barh(df_resultado['name'], df_resultado['media_pedidos'], color='skyblue', edgecolor='black')

plt.title('10 Campanhas com Maior Média de Pedidos por Cliente', fontsize=16)
plt.xlabel('Média de Pedidos por Cliente', fontsize=12)
plt.ylabel('Nome da Campanha', fontsize=12)
plt.gca().invert_yaxis()
plt.grid(axis='x', linestyle='--', alpha=0.6)

for i, v in enumerate(df_resultado['media_pedidos']):
    plt.text(v + 0.05, i, f'{v:.2f}', color='black', va='center', fontsize=10)

plt.tight_layout()
plt.show()

df_resultado.to_csv('top10_media_pedidos_por_campanha.csv', sep=';', index=False)

print("Arquivo salvo: top10_media_pedidos_por_campanha.csv")

Pedidos Por Região (criado)

In [ ]:
# @title
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

sns.set_style("whitegrid")

pedidos_por_regiao = df_pedidos['customerRegion'].value_counts().reset_index()
pedidos_por_regiao.columns = ['Regiao', 'Total_Pedidos']

df_regiao_top10 = pedidos_por_regiao.head(10)

print("Pedidos por Região")
print("-" * 30)
print(df_regiao_top10.to_string(index=False))

plt.figure(figsize=(10, 6))
ax = sns.barplot(
    x='Total_Pedidos',
    y='Regiao',
    hue='Regiao',
    data=df_regiao_top10,
    palette='Spectral',
    legend=False
)

plt.title('Top 10 Regiões por Volume de Pedidos', fontsize=16)
plt.xlabel('Número Total de Pedidos', fontsize=12)
plt.ylabel('Região', fontsize=12)

for p in ax.patches:
    ax.annotate(
        f'{int(p.get_width())}',
        (p.get_width(), p.get_y() + p.get_height() / 2),
        ha='left', va='center',
        xytext=(5, 0), textcoords='offset points'
    )
plt.subplots_adjust(left=0.25, right=0.95, top=0.9, bottom=0.1)
plt.show()


In [ ]:
# @title
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

sns.set_theme(style="whitegrid")
pd.set_option('display.max_columns', None)

print("Análise de pedidos por região\n")

df_pedidos = pd.read_csv('Order_semicolon.csv', sep=';')

print(f"Dados carregados!")
print(f"Total de registros: {df_pedidos.shape[0]}")
print(f"Colunas disponíveis: {list(df_pedidos.columns)}\n")

df_pedidos = df_pedidos[['id', 'customer', 'customerRegion', 'totalAmount']]
df_pedidos.drop_duplicates(inplace=True)
df_pedidos.dropna(subset=['customerRegion'], inplace=True)
df_pedidos['totalAmount'] = pd.to_numeric(df_pedidos['totalAmount'], errors='coerce').fillna(0)

print("Dados limpos e prontos!\n")

print("Total de pedidos por região\n")

pedidos_por_regiao = df_pedidos['customerRegion'].value_counts().reset_index()
pedidos_por_regiao.columns = ['Regiao', 'Total_Pedidos']
df_regiao_top10 = pedidos_por_regiao.head(10)

print("===== Pedidos por Região (Top 10) =====")
print(df_regiao_top10.to_string(index=False))
print("=======================================\n")

print("📊 Gerando gráfico do Top 10 de regiões...\n")

plt.figure(figsize=(10, 6))
ax = sns.barplot(
    x='Total_Pedidos',
    y='Regiao',
    hue='Regiao',
    data=df_regiao_top10,
    palette='Spectral',
    legend=False
)

plt.title('Top 10 Regiões por Volume de Pedidos', fontsize=16)
plt.xlabel('Número Total de Pedidos', fontsize=12)
plt.ylabel('Região', fontsize=12)

for p in ax.patches:
    ax.annotate(f'{int(p.get_width())}',
                (p.get_width(), p.get_y() + p.get_height() / 2),
                ha='left', va='center', xytext=(5, 0), textcoords='offset points')

plt.subplots_adjust(left=0.25, right=0.95, top=0.9, bottom=0.1)
plt.show()

df_regiao_top10.to_csv('top10_regioes_por_pedidos.csv', sep=';', index=False)

print("Arquivo salvo: top10_regioes_por_pedidos.csv")
print("Processo concluído com sucesso!")

Faixa Etária (criado)

In [ ]:
# @title
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

bins = [18, 23, 28, 33, 38, 43, 48, 53, 58, 63]
labels = ['18-22', '23-27', '28-32', '33-37', '38-42', '43-47', '48-52', '53-57', '58-65']

df_pedidos['Faixa_Etaria'] = pd.cut(df_pedidos['customerAge'], bins=bins, labels=labels, right=True, include_lowest=True)

pedidos_por_idade = df_pedidos['Faixa_Etaria'].value_counts(sort=False).reset_index()
pedidos_por_idade.columns = ['Faixa_Etaria', 'Total_Pedidos']

print("\nPedidos por Faixa Etária")
print("-" * 30)
print(pedidos_por_idade.to_string(index=False))

plt.figure(figsize=(10, 6))
ax = sns.barplot(
    x='Faixa_Etaria',
    y='Total_Pedidos',
    data=pedidos_por_idade,
    palette='Blues_d'
)

plt.title('Distribuição de Pedidos por Faixa Etária', fontsize=16)
plt.xlabel('Faixa Etária (Anos)', fontsize=12)
plt.ylabel('Número Total de Pedidos', fontsize=12)
plt.xticks(rotation=45)

for p in ax.patches:
    ax.annotate(f'{int(p.get_height())}',
                (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', fontsize=10, color='black', xytext=(0, 5), textcoords='offset points')

plt.tight_layout()
plt.show()

In [ ]:
# @title
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")
plt.style.use('seaborn-v0_8-whitegrid')

df_pedidos = pd.read_csv('Order_semicolon.csv', sep=';')

print(f"Total de registros: {df_pedidos.shape[0]}")
print(f"Colunas disponíveis: {list(df_pedidos.columns)}\n")

df_pedidos.drop_duplicates(inplace=True)
df_pedidos = df_pedidos[df_pedidos['customerAge'].notnull()]
df_pedidos = df_pedidos[df_pedidos['customerAge'].between(18, 65)]

df_pedidos['customerAge'] = pd.to_numeric(df_pedidos['customerAge'], errors='coerce').fillna(0)

print("Criando faixas etárias para análise...\n")

bins = [18, 23, 28, 33, 38, 43, 48, 53, 58, 65]
labels = ['18-22', '23-27', '28-32', '33-37', '38-42', '43-47', '48-52', '53-57', '58-65']

df_pedidos['Faixa_Etaria'] = pd.cut(
    df_pedidos['customerAge'],
    bins=bins,
    labels=labels,
    right=True,
    include_lowest=True
)

pedidos_por_idade = df_pedidos['Faixa_Etaria'].value_counts(sort=False).reset_index()
pedidos_por_idade.columns = ['Faixa_Etaria', 'Total_Pedidos']

print("===== Pedidos por Faixa Etária =====")
print(pedidos_por_idade.to_string(index=False))
print("====================================\n")

plt.figure(figsize=(10, 6))
ax = sns.barplot(
    x='Faixa_Etaria',
    y='Total_Pedidos',
    data=pedidos_por_idade,
    palette='Blues_d'
)

plt.title('Pedidos por Faixa Etária', fontsize=16)
plt.xlabel('Faixa Etária (Anos)', fontsize=12)
plt.ylabel('Número Total de Pedidos', fontsize=12)
plt.xticks(rotation=45)

for p in ax.patches:
    ax.annotate(f'{int(p.get_height())}',
                (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', fontsize=10, color='black',
                xytext=(0, 5), textcoords='offset points')

plt.tight_layout()
plt.show()

df_pedidos[['customerAge', 'Faixa_Etaria']].to_csv('clientes_faixa_etaria.csv', sep=';', index=False)

print("Arquivo salvo: clientes_faixa_etaria.csv")
print("Processo concluído com sucesso!")